# Recalculating TLEs for delayed launches

When a launch is delayed, the orbit moves in relation to the earth.
This can be described by a shift in the epoch and the right ascension of the ascending node in the TLE.

In [36]:
%matplotlib inline

from datetime import datetime, timedelta
from pytz import timezone
from skyfield.api import EarthSatellite, load, wgs84, N, S, E, W

from sgp4.io import twoline2rv
from sgp4.earth_gravity import wgs84

from astropy.coordinates import Longitude, Angle
from astropy import units as u
from astropy.time import Time

import sys
path = r'C:\Users\Virgil MESLE\Documents\0-Obsidian\Python\My_codes\0-My_module'
sys.path.append(path)
import fonctions_csum as fctcsum

In [37]:
def gmst(t):
    return Time(t).sidereal_time('mean', 'greenwich')

def fractional_days(t):
    d = t - datetime(t.year, 1, 1)
    return t.timetuple().tm_yday + (d.seconds + d.microseconds/1e6)/(60*60*24)

In [38]:
def print_tle(tle, new_epoch, new_nodeo):
    def checksum(proto_tle):
        s = 0
        for c in proto_tle:
            if c.isdigit():
                s += int(c)
            if c == '-':
                s += 1
        return s%10

    tle0,old1,old2 = tle
    tle1_proto = '{} {:2d}{:.8f} {}'.format(old1[:17],
                              abs(new_epoch.year)%100,
                              fractional_days(new_epoch),
                              old1[33:-1])
    tle2_proto = '{} {:>8.4f} {}'.format(old2[:16],
                                   Angle(new_nodeo*u.radian).degree,
                                   old2[26:68])
            
    return (tle0,
            tle1_proto + str(checksum(tle1_proto)),
            tle2_proto + str(checksum(tle2_proto)))

def launch_tle(tle, launch_date, new_launch_date):
    sat = twoline2rv(tle[1], tle[2], whichconst=wgs84)

    # New Epoch
    new_epoch = sat.epoch - launch_date + new_launch_date

    # New Right ascension of ascending node in radians
    new_nodeo = (gmst(new_launch_date) - gmst(launch_date) + Longitude(sat.nodeo * u.radian)).rad
    
    return print_tle(tle, new_epoch, new_nodeo)

In [39]:
# From https://community.libre.space/t/electron-its-business-time-launch-this-weekend-irvine-01-amateur-payload/2819/4
tle = ["TLE_CUSTOM",
       "1 58470U 23185J   24053.15047623  .00011905  00000-0  57071-3 0  9994",
       "2 58470  97.4426 121.4026 0013052 330.1961  29.8528 15.18987494 12506"]


# launch_date_ref = datetime(2018, 11, 11, 3, 0)
launch_date_ref = datetime.now()
new_launch_date = datetime.now() + timedelta(hours = 5.3 )
# new_launch_date = datetime(2018, 11, 11, 4, 5)

new_tle = launch_tle(tle, launch_date_ref, new_launch_date)

print('New TLE for a launch delay of {}h :\n'.format(new_launch_date - launch_date_ref))
print(new_tle[0]+'-delayed')
print(new_tle[1])
print(new_tle[2])

TLE = f'{new_tle[1]}\n{new_tle[2]}'
TLE

New TLE for a launch delay of 5:18:00h :

TLE_CUSTOM-delayed
1 58470U 23185J   2453.37130956  .00011905  00000-0  57071-3 0  9990
2 58470  97.4426 201.1203 0013052 330.1961  29.8528 15.18987494 12509


'1 58470U 23185J   2453.37130956  .00011905  00000-0  57071-3 0  9990\n2 58470  97.4426 201.1203 0013052 330.1961  29.8528 15.18987494 12509'

In [40]:
ts = load.timescale()
sat = fctcsum.TLE_to_sat(TLE)

In [33]:
test = '1 58470U 23185J   24054.12859687  .00012218  00000-0  58518-3 0  9999\n2 58470  97.4424 122.3758 0012991 326.4557  33.5854 15.19012117 12658'
test_sat = fctcsum.TLE_to_sat(test)

In [34]:
fctcsum.satellite_pass(test_sat,dt,  2)

,Date(FR),Event,Elevation,Illumination
0,23 Feb 2024 - 09:22:45,rise above 0,0.0,in sunlight
1,23 Feb 2024 - 09:25:54,culminate,3.5,in sunlight
2,23 Feb 2024 - 09:29:02,set below 0,-0.0,in sunlight
3,23 Feb 2024 - 10:54:40,rise above 0,0.0,in sunlight
4,23 Feb 2024 - 11:00:33,culminate,63.6,in sunlight
5,23 Feb 2024 - 11:06:23,set below 0,-0.0,in sunlight
6,23 Feb 2024 - 12:29:21,rise above 0,0.0,in sunlight
7,23 Feb 2024 - 12:33:56,culminate,10.8,in sunlight
8,23 Feb 2024 - 12:38:31,set below 0,-0.0,in sunlight
9,23 Feb 2024 - 20:06:10,rise above 0,0.0,in shadow


In [46]:
dt = datetime(2024,2,23,3,15,0)

In [8]:
#### SAT EPOCH IN JULIAN DATE

In [48]:
sat.epoch = ts.from_datetime(timezone('UTC').localize(dt))
sat.epoch ## Julian date

<Time tt=2460363.6362174074>

In [19]:
fctcsum.satellite_pass(sat,dt,  2)

,Date(FR),Event,Elevation,Illumination
0,23 Feb 2024 - 03:23:56,rise above 0,0.0,in shadow
1,23 Feb 2024 - 03:29:41,culminate,80.8,in shadow
2,23 Feb 2024 - 03:35:32,set below 0,-0.1,in shadow
3,23 Feb 2024 - 05:00:06,rise above 0,0.0,in shadow
4,23 Feb 2024 - 05:04:07,culminate,6.9,in shadow
5,23 Feb 2024 - 05:08:12,set below 0,-0.1,in shadow
6,23 Feb 2024 - 14:54:15,rise above 0,0.2,in sunlight
7,23 Feb 2024 - 14:58:12,culminate,6.7,in sunlight
8,23 Feb 2024 - 15:02:14,set below 0,-0.0,in sunlight
9,23 Feb 2024 - 16:26:46,rise above 0,0.0,in sunlight


In [30]:
## A VOIRRRR POUR CHECK NOUVELLE TLE
tle0,old1,old2 = tle
tle1_proto = '{} {:2d}{:.8f} {}'.format(old1[:17],
                            abs(new_epoch.year)%100,
                            fractional_days(new_epoch),
                            old1[33:-1])
tle2_proto = '{} {:>8.4f} {}'.format(old2[:16],
                                Angle(new_nodeo*u.radian).degree,
                                old2[26:68])

## Testing the Implementation against 'launchtle' from sattools


In [41]:
launch_date_ref = datetime(2018, 11, 11, 3, 0)
tle_ref = ["1 70002U 18599A   18315.16151858  .00000000  00000-0  00000-0 0    07",
           "2 70002  85.1205  90.1568 0012705 292.5520 107.9249 15.20792276    04"]

# launchtle -c irvine.txt -i 70002 -t 2018-11-11T03:00:00 -T 2018-11-11T03:00:00 -I 70002 -d 18599A
fixtures = [(datetime(2018, 11, 11, 3, 0),
            ["1 70002U 18599A   18315.16151858  .00000000  00000-0  00000-0 0    07",
             "2 70002  85.1205  90.1568 0012705 292.5520 107.9249 15.20792276    04"]),
            (datetime(2018, 11, 11, 3, 5),
            ["1 70002U 18599A   18315.16499080  .00000000  00000-0  00000-0 0    09",
             "2 70002  85.1205  91.4102 0012705 292.5520 107.9249 15.20792276    02"]),
            (datetime(2018, 11, 11, 4, 0),
            ["1 70002U 18599A   18315.20318525  .00000000  00000-0  00000-0 0    08",
             "2 70002  85.1205 105.1979 0012705 292.5520 107.9249 15.20792276    07"]),
            (datetime(2018, 11, 12, 3, 0),
            ["1 70002U 18599A   18316.16151858  .00000000  00000-0  00000-0 0    08",
             "2 70002  85.1205  91.1424 0012705 292.5520 107.9249 15.20792276    06"])]

In [42]:
for new_launch_date, tle_correct in fixtures:
    tle = launch_tle(['DUMMYSAT', *tle_ref], launch_date_ref, new_launch_date)
    assert(tle_correct[1] == tle[2])
    assert(tle_correct[0] == tle[1])

--> For the very limited set of examples this implemenation yields the same result as 
     the program 'launchtle' from Cees Bassa's sattools.